In [157]:
import pandas as pd
import numpy as np
import random
from scipy.io import loadmat
import scipy.optimize as opt

from multiprocessing import Pool
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report#这个包是评价报告
from typing import List,Tuple
import plotly.express as px
import plotly.graph_objects as go

In [158]:
# g(theta,x)
def sigmoid(x):
    s = 1 / (1 + np.exp(-x))
    return s

def serialize(*arrays:np.ndarray):
    arr_shapes:List[tuple] = []
    arr_flatten = np.array([])
    for array in arrays:
        arr_shapes.append(array.shape)
        arr_flatten = np.hstack([arr_flatten,array.flatten()])
    return arr_flatten,arr_shapes

def deserialize(array:np.ndarray, shapes:List[tuple]):
    position = 0
    arrays:List[np.ndarray] = []
    for shape in shapes:
        length = np.prod(shape)
        arrays.append(array[position:position+length].reshape(*shape))
        position += length
    return arrays

def forward_propa(thetas:List[np.ndarray],X:np.ndarray):
    n_layers =len(thetas)+1
    a_list = []
    z_list = []

    a_list.append(np.insert(X,0,1,axis=1))
    for index,theta in enumerate(thetas):
        z = a_list[index]@theta.T
        z_list.append(z)
        if index+1 == n_layers-1: # output layer
            a_list.append(sigmoid(z))
        else:
            a_list.append(np.insert(sigmoid(z),0,1,axis=1))
    return a_list,z_list

def h(thetas,X):
    return forward_propa(thetas,X)[0][-1]
    
# J(Theta)
def cost(thetas_flat:np.ndarray, shapes:List[tuple],X:np.ndarray,y:np.ndarray):
    theta = deserialize(thetas_flat,shapes)
    return np.mean(-np.sum(np.log(h(theta,X))*y,axis=1)-np.sum((1-y)*np.log(1-h(theta,X)),axis=1))

def regularized_cost(thetas_flat:np.ndarray, shapes:List[tuple], X:np.ndarray, y:np.ndarray, L:float=1.):
    n = X.shape[0]
    regular_term = 0
    theta = deserialize(thetas_flat,shapes)
    for t in theta:
        regular_term += np.sum(t[:,1:]**2)
    regular_term = regular_term/(2*n)*L
    return cost(thetas_flat,shapes,X,y)+regular_term

def sigmoid_gradient(x):
    return sigmoid(x)*(1-sigmoid(x))

# def gradients(theta,x,y):
#     return np.mean(x.T*(h(theta,x)-y),axis=1)

In [159]:
data = loadmat('ex4data1.mat')
thetas = loadmat("ex4weights.mat")
theta1 = thetas["Theta1"]
theta2 = thetas["Theta2"]
thetas = [theta1,theta2]
thetas_flat,shapes = serialize(theta1,theta2)
X = data["X"]
y = data["y"]

In [160]:
encoder_y = OneHotEncoder(sparse=False)
y_onehot = encoder_y.fit_transform(y)

In [161]:
# 前100个 排列成10*10 (200*200)

digits = [X[random.randint(0,4999)].reshape(20,20).T for _ in range(100)]

rows_digits = []
for row in range(10):
    rows_digits.append(np.hstack(digits[row*10:row*10+10]))

digits_img = np.vstack(rows_digits)

In [162]:
fig = px.imshow(digits_img,color_continuous_scale='gray')
fig.show()

In [8]:
# # 模拟H的计算过程，不需要了
# a1 = np.insert(X,0,1,axis=1)
# z2 = a1@theta1.T
# a2 = sigmoid(z2)
# a2 = np.insert(a2,0,1,axis=1)
# z3 = a2@theta2.T
# a3 = sigmoid(z3)
# np.mean(-np.sum(np.log(a3)*y_onehot,axis=1)-np.sum((1-y_onehot)*np.log(1-a3),axis=1))

In [163]:
cost(thetas_flat,shapes,X,y_onehot)

0.2876291651613189

In [164]:
regularized_cost(thetas_flat,shapes,X,y_onehot)

0.38376985909092365

# Back Propagation

In [179]:
epsilon = 0.12
theta1_init = np.random.rand(25,401) * 2 * epsilon - epsilon
theta2_init = np.random.rand(10,26) * 2 * epsilon - epsilon
thetas_flat,shapes = serialize(theta1_init,theta2_init)
thetas=deserialize(thetas_flat,shapes)

In [166]:
def gradients(thetas_flat,shapes,X,y_onehot):
    thetas = deserialize(thetas_flat,shapes)
    deltas = {} 
    big_deltas={index:np.zeros(theta.shape) for index,theta in enumerate(thetas)}

    a,z = forward_propa(thetas,X)
    # X:5000*400
    # y_onehot:5000*10
    # a[0]:5000*401
    # a[1]:5000*26
    # a[2]:5000*10 h(x)
    # z[0]:5000*25
    # z[1]:5000*10
    # theta[0]:25*401
    # theta[1]:10*26
    # delta[1]:5000*10
    # delta[0]:5000*25
    # big_delta[0]:25*401
    # big_delta[1]:10*26

    for index,theta in reversed(list(enumerate(thetas))):
        if np.array_equal(theta,thetas[-1]): # 输出层 
            deltas[index] = a[index+1]-y_onehot #
        else:
            z_temp = np.insert(z[index],0,1,axis=1)
            deltas_temp = deltas[index+1]@thetas[index+1]*sigmoid_gradient(z_temp)
            deltas[index] = deltas_temp[:,1:]

    for key,delta in deltas.items():
        big_deltas[key] = (big_deltas[key] + delta.T@a[key])/(X.shape[0])
    
    big_deltas=[big_deltas[index] for index in range(len(thetas))]
    
    return serialize(*big_deltas)

## gradient checking

In [167]:
def expand_array(arr):
    """replicate array into matrix
    [1, 2, 3]

    [[1, 2, 3],
     [1, 2, 3],
     [1, 2, 3]]
    """
    # turn matrix back to ndarray
    return np.array(np.matrix(np.ones(arr.shape[0])).T @ np.matrix(arr))

In [169]:
eps = 10**-4
thetas_matrix = expand_array(thetas_flat)
epsilon_matrix = np.identity(len(thetas_flat)) * eps
plus_matrix = thetas_matrix + epsilon_matrix
minus_matrix = thetas_matrix - epsilon_matrix
gradient_numerical = []
for i in range(len(thetas_flat)):
    plus = plus_matrix[i]
    minus = minus_matrix[i]
    gradient_numerical.append((cost(plus,shapes,X,y_onehot) - cost(minus,shapes,X, y_onehot)) / (eps * 2))


In [182]:
gradient_analytical,_ = gradients(thetas_flat,shapes,X,y_onehot)

In [183]:
gradient_analytical

array([-0.0196655 ,  0.        ,  0.        , ...,  0.16050616,
        0.16911284,  0.14415814])

In [171]:
diff = np.linalg.norm(gradient_numerical - gradient_analytical) / np.linalg.norm(gradient_numerical + gradient_analytical)


In [172]:
diff

7.869349676722695e-11

## 2.5  Regularized Neural Networks

In [173]:
def regularized_gradient(thetas_flat,shapes,X,y,l=1):
    gradient_analytical,_ = gradients(thetas_flat,shapes,X,y)
    regular_term = thetas_flat/X.shape[0]*l
    return gradient_analytical+regular_term

In [174]:
regularized_gradient(thetas_flat,shapes,X,y_onehot)

array([ 3.30953378e-03, -1.39536200e-06, -1.07102726e-05, ...,
        2.07291138e-01,  2.07566474e-01,  1.68694188e-01])

## 2.6  Learning parameters usingfmincg

In [175]:
def nn_training(thetas_flat,shapes,X, y):
    """regularized version
    the architecture is hard coded here... won't generalize
    """

    res = opt.minimize(fun=regularized_cost,
                       x0=thetas_flat,
                       args=(shapes,X, y,1),
                       method='TNC',
                       jac=regularized_gradient,
                       options={'maxiter': 400})
    return res

In [184]:
res = nn_training(thetas_flat,shapes,X, y_onehot)#慢

In [185]:
res

     fun: 0.3236950221686778
     jac: array([-6.01990570e-05,  3.05054746e-08,  3.00255561e-08, ...,
        2.77671971e-05, -9.29675026e-05, -1.31090399e-05])
 message: 'Max. number of function evaluations reached'
    nfev: 400
     nit: 27
  status: 3
 success: False
       x: array([-5.43751062e-02,  1.52527373e-04,  1.50127781e-04, ...,
       -1.08009611e+00,  6.72082639e-01,  1.89927574e+00])

In [186]:
final_theta = res.x

In [188]:
final_thetas = deserialize(final_theta,shapes)

In [189]:
a_s,z_s = forward_propa(final_thetas,X)

In [193]:
y_pred = np.argmax(a_s[-1], axis=1) + 1

In [195]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           1       0.99      0.99      0.99       500
           2       0.99      0.99      0.99       500
           3       0.99      0.99      0.99       500
           4       1.00      0.99      0.99       500
           5       0.99      1.00      1.00       500
           6       1.00      1.00      1.00       500
           7       0.99      0.99      0.99       500
           8       1.00      1.00      1.00       500
           9       0.99      0.99      0.99       500
          10       1.00      1.00      1.00       500

    accuracy                           0.99      5000
   macro avg       0.99      0.99      0.99      5000
weighted avg       0.99      0.99      0.99      5000

